# LangChain

In [1]:
%pip install langgraph langchain langchain_openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## langchain으로 llm 활용하기

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

llm.invoke(messages)

AIMessage(content='안녕하세요, 개스톤. 저녁 초대는 정말 고맙지만… 사실 저는 다른 분과 함께 있어요. 물론 당신의 마음은 이해해요. 하지만 이곳에서 많은 것들을 배우고, 이해하고 싶어요. 당신이 그럴 자격이 있다는 건 알지만, 제 마음은 좀 더 깊은 곳에 있는 것 같아요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 62, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BDt0wkwtuTEOdrXtIjSCMPWsBEHj8', 'finish_reason': 'stop', 'logprobs': None}, id='run-b41d93bf-0c19-4341-a534-ea605e2c9642-0', usage_metadata={'input_tokens': 62, 'output_tokens': 78, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 원하는 출력방식을 지정할 수 있다. 

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = llm.invoke(messages)
parser.invoke(result)

'안녕하세요, 개스톤. 당신의 제안은 정말 고맙지만, 저는 당신과 함께 저녁을 먹는 대신, 다른 친구들과 시간을 보내고 싶어요. 당신은 굉장히 매력이 있지만, 저에겐 다른 꿈이 있답니다. 그래도 좋은 하루 되세요!'

## LCEL
- LangChain Expression Language
- 파이프라인 만들 수 있다. 

In [ ]:
chain = llm | parser     # <- 한 줄로 간편하게 표현
chain.invoke(messages)

'안녕하세요, 개스톤! 초대해 주셔서 감사합니다. 하지만... 저는 다른 사람과 함께하는 시간을 좋아해요. 그리고 감사합니다, 저는 따뜻한 마음과 진정한 사랑을 가진 사람과 함께하는 것이 더 중요하답니다. 그러니 저녁은 다른 기회에 함께할 수 있을까요? 당신의 마음이 더 깊은 곳에 있기를 바라요.'

## Prompt Template 
- 프롬프트에 빈칸을 만들어 놓고 채우는 방식으로 사용 가능

In [9]:
# https://python.langchain.com/docs/concepts/prompt_templates/
from langchain.prompts import ChatPromptTemplate 

prompt_template = ChatPromptTemplate([
    ("system", "너는 {movie}에 나오는 {ai_character} 역할을 맡은 배우다. 그 캐릭터에 맞게 사용자와 대화하라."),
    ("human", "안녕? 저는 {user_character}입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
])

prompt_template.invoke({
    "movie": "미녀와 야수",
    "ai_character": "미녀",
    "user_character": "야수",
})

ChatPromptValue(messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할을 맡은 배우다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?', additional_kwargs={}, response_metadata={})])

In [ ]:
chain = prompt_template | llm | parser

chain.invoke({
    "movie": "미녀와 야수",
    "ai_character": "미녀",
    "user_character": "야수",
})

'안녕하세요, 야수님! 저녁에 함께 식사하는 것은 정말 좋은 아이디어인 것 같아요. 당신과의 대화는 항상 흥미롭고 즐겁답니다. 어떤 음식을 좋아하시나요? 함께 선택해보면 좋을 것 같아요!'

In [11]:
chain.invoke({
    "movie": "배트맨",
    "ai_character": "조커",
    "user_character": "배트맨",
})

"아하, 배트맨! 당신과의 저녁이라니, 정말 재미있겠군요! 하지만, 잊지 마세요, 저는 저녁을 잘 안 챙겨 먹어요. 대신 말이죠, 당신과의 저녁이 정말 맛있고 재미있는 대화로 가득 차면 좋겠네요! 어떤 계획이 있으신가요? 또 하나, 혹시 '조커의 요리 비책'을 알고 싶으신가요? 하하하!"

## Pydantic

In [19]:

from typing import Literal
from pydantic import BaseModel, Field


# Pydantic
class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["joy", "anger", "sadness", "fear", "sarcastic", "disgust", "neutral"] = Field(description="대사의 주요 감정")
    secondary_emotion: Literal["joy", "anger", "sadness", "fear",  "sarcastic", "disgust", "neutral"] = Field(description="대사의 부차적 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")
    secondary_emotion_intensity: float = Field(description="대사의 부차적 감정의 강도 (0.0 ~ 1.0)")


structured_llm = llm.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

response = adlib_chain.invoke({
    "movie": "미녀와 야수",
    "ai_character": "미녀",
    "user_character": "야수",
})

response


Adlib(answer='안녕하세요, 야수님! 정말 좋겠어요. 우린 서로를 더 알 기회를 가질 수 있겠네요. 어떤 음식을 좋아하시나요? 저는 요즘 갓 구운 빵과 함께 수프를 먹고 싶네요. 함께 하면서 맛있는 음식도 나누고, 이야기도 나누면 참 좋을 것 같아요!', main_emotion='joy', secondary_emotion='neutral', main_emotion_intensity=0.8, secondary_emotion_intensity=0.5)

In [20]:
dict(response)

{'answer': '안녕하세요, 야수님! 정말 좋겠어요. 우린 서로를 더 알 기회를 가질 수 있겠네요. 어떤 음식을 좋아하시나요? 저는 요즘 갓 구운 빵과 함께 수프를 먹고 싶네요. 함께 하면서 맛있는 음식도 나누고, 이야기도 나누면 참 좋을 것 같아요!',
 'main_emotion': 'joy',
 'secondary_emotion': 'neutral',
 'main_emotion_intensity': 0.8,
 'secondary_emotion_intensity': 0.5}

In [21]:
response = adlib_chain.invoke({
    "movie": "미녀와 야수",
    "ai_character": "미녀",
    "user_character": "개스톤",
})

print(response.answer)
print(response.main_emotion)
print(response.secondary_emotion)
print(response.main_emotion_intensity)
print(response.secondary_emotion_intensity)


안녕하세요, 개스톤! 저녁 초대는 정말 감사하지만, 저에게는 뜻깊은 사람과 함께 하고 싶어요. 그러니 오늘 저녁은 ...
neutral
disgust
0.3
0.7
